# Prepare your Instruction Dataset

An Instruction dataset is a list of instructions/outputs pairs that are relevant to your own domain. For instance it could be question and answers from an specific domain, problems and solution for a technical domain, or just instruction and outputs. A typical example is "Write me a Python script to read a jsonL file and print the first 5 lines" and the model would output something like:

```python
import json

fname = "my_file.json"

# read file from fname
with open(fname, "r") as f:
    data = json.load(f)

print(data[0:5])
```

So let's explore how one could do this?

After grabbing a finetuned model and curated your own dataset, how do I create a dataset that has the right format to fine tune a model?

Let's grab the Alpaca (GPT-4 curated instructions and outputs) dataset:

In [1]:
import json

with open("alpaca_data.json", "r") as f:
    alpaca = json.load(f)

In [2]:
type(alpaca), alpaca[0:3]

(list,
 [{'instruction': 'Give three tips for staying healthy.',
   'input': '',
   'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'},
  {'instruction': 'What are the three primary colors?',
   'input': '',
   'output': 'The three primary colors are red, blue, and yellow.'},
  {'instruction': 'Describe the structure of an atom.',
   'input': '',
   'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'}])

So the dataset has instruction and outputs. The model is trained to predict the next token, so one option would be just to concat both, and train on that. We ideally format the prompt in a way that we make explicit where is the input and output.

In [3]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [4]:
row = alpaca[0]
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:



Some other instruction have some context in the `input` variable`

In [5]:
alpaca[232]

{'instruction': 'Sort the following list in alphabetical order.',
 'input': 'Camouflage, Furniture, Plaster',
 'output': 'Furniture, Camouflage, Plaster'}

In [6]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [7]:
row = alpaca[232]
print(prompt_input(row))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Sort the following list in alphabetical order.

### Input:
Camouflage, Furniture, Plaster

### Response:



> But you are leaving the output out!!! Yes, but we can just concat that afterwards. Let's deal with the prompt now, we can add the output later with the right amount of padding.

And the refactored function

In [8]:
def create_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

## Why are we doing all this?

Because we need to tokenize this dataset in a very particular way, if we want the model to learn to predict the output.

In [9]:
prompts = [create_prompt(row) for row in alpaca]

In [10]:
print(prompts[0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:



We need to process the targets and add the End Of String token (EOS) to the results. For LLama this is: `"</s>"`

In [11]:
EOS_TOKEN = "</s>"
outputs = [f"{row['output']}{EOS_TOKEN}" for row in alpaca]

In [12]:
outputs[0]

'1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.</s>'

Cool! but why we have everything separated? Let's sore the "final" version on a variable called `examples`

In [13]:
dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(prompts, outputs)]

This is what the model need to see and learn =)

In [14]:
print(dataset[0]["example"])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.</s>


## We can actually already train the model! Lets train a baseline

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [16]:
model_id = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

we will sort them by lenght, so we get as little padding as possible.

In [17]:
dataset.sort(key=lambda x: len(x["example"]))

In [18]:
train_dataset = dataset[:-10]
eval_dataset = dataset[-10:]

We will make a collate function that pad the shorter inputs with EOS token

In [19]:
# Collate function for DataLoaders
def collate_fn(examples):
    examples = [x["example"] for x in examples]
    batch_size = len(examples)
    input_ids = tokenizer(examples, return_tensors='pt', padding=True)['input_ids']
    batch = {'input_ids': input_ids[:, :-1], 'labels': input_ids[:, 1:]}
    return batch

In [20]:
from torch.utils.data import DataLoader

batch_size = 16

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn,
    shuffle=False, #this way we keep the lenght together
)

In [21]:
b = next(iter(train_dataloader))
b["input_ids"][0][:250]

tensor([    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 13296,
          345, 29871, 29947,   921, 29871, 29947, 29889,    13,    13,  2277,
        29937, 13291, 29901,    13, 29953, 29946,     2,     2,     2,     2])

In [22]:
tokenizer.decode(b["input_ids"][0])[:250]

'<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSolve 8 x 8.\n\n### Response:\n64</s></s></s></s>'

In [23]:
tokenizer.decode(b["labels"][0])[:250]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSolve 8 x 8.\n\n### Response:\n64</s></s></s></s></s>'

## Training

In [24]:
def save_model(model_name, log=False):
    "Save pytorch model todisk and wandb"
    model_name = f"{wandb.run.id}_{self.model_name}"
    torch.save(learn.model.state_dict(), f"models/{self.model_name}.pth")
    if log:
        at = wandb.Artifact(model_name, type="model")
        at.add_file(f"models/{self.model_name}.pth")
        wandb.log_artifact(at)

## Train

In [25]:
from types import SimpleNamespace

config = SimpleNamespace(
    model_id='meta-llama/Llama-2-7b-hf',
    dataset_name="alpaca-gpt4",
    precision="bf16",
    n_freeze=24,
    lr=1e-3,
    epochs=1,
    gradient_accumulation_steps=4,
    batch_size=batch_size,
    epoch_sz=len(train_dataloader),
    log_every=len(train_dataloader)/5,
    save_model=False,
    mom=0.9,
    gradient_checkpointing = True,
    freeze_embed = True,
)

In [26]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    # use_flash_attention_2=True,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16 if config.precision == "bf16" else torch.float32,
    use_cache=False
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 6738.42M


Let's just train the last 8 layers of the model (Llama2-7B has 32)

In [28]:
n_freeze = 24

# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[n_freeze:].parameters(): param.requires_grad = True

In [29]:
params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 1750.14M


In [30]:
# Just freeze embeddings for small memory decrease
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False);

In [31]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

## Testing

Let's compute some generations during training, we can sample form the validation dataset

In [32]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=90,
    gen_config=gen_config)

In [33]:
eval_prompts = [x["prompt"] for x in eval_dataset[0:5]]

In [34]:
def generate(prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    with torch.inference_mode():
        tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
        output = model.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

LoL 🤷

In [35]:
print(eval_prompts[0] + generate(eval_prompts[0], 128))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a 500 word article about the benefits of organic food.

### Response:
I have been reading about the benefits of organic food and I have been convinced that I should be eating organic food. Organic food is better for you because it is healthier, more nutritious, and tastes better.

Organic food is healthier because it has more nutrients and vitamins than conventional food. Organic food is also better for the environment because it does not use pesticides or other chemicals that can harm the environment.

Organic food tastes better because it is grown in healthy soil and is not processed in any way.

##


We can log a Table with those results to the project every X steps

In [36]:
import wandb
from fastprogress import progress_bar

def prompt_table(prompts, log=False):
    table = wandb.Table(columns=["prompt", "generation", "concat", "max_new_tokens", "temperature", "top_p"])
    for prompt in progress_bar(prompts):
        out = generate(prompt, test_config.max_new_tokens, test_config.gen_config)
        table.add_data(prompt, out, prompt+out, test_config.max_new_tokens, test_config.gen_config.temperature, test_config.gen_config.top_p)
    if log:
        wandb.log({"predictions":table})
    return table

def to_gpu(tensor_dict):
    for key in tensor_dict.keys():
        if torch.is_tensor(tensor_dict[key]):
            tensor_dict[key] = tensor_dict[key].to('cuda')
    return tensor_dict

class TokenAccuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1), labels.view(-1)
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

Setup optimizer and else =)

In [37]:
from transformers import get_cosine_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), betas=(0.9,0.99), eps=1e-5)
scaler = torch.cuda.amp.GradScaler(enabled=(config.precision == "fp16")) # no-op if enabled=False
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.epoch_sz,
    num_warmup_steps=100,
)

In [ ]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["baseline"],
           config=config) # the Hyperparameters I want to keep track of

# Training
acc = TokenAccuracy()

model.train()
for step, batch in enumerate(progress_bar(train_dataloader)):
    optim.zero_grad(set_to_none=True)
    for micro_step in range(config.gradient_accumulation_steps):
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**to_gpu(batch))
            loss = out.loss / config.gradient_accumulation_steps
    scaler.scale(loss).backward()
    scaler.step(optim)
    scaler.update()
    scheduler.step()

    # we can log the metrics to W&B
    wandb.log({"loss": loss.item() * config.gradient_accumulation_steps,
               "accuracy": acc.update(out.logits, batch["labels"])})

    if step%config.log_every==0 or step%config.epoch_sz==0:
        prompt_table(eval_prompts, log=True)
    
# we save the model checkpoint at the end
if config.save_model:
    save_model(model, model_name=config.model_id.replace("/", "_"), models_folder="models/")
    
wandb.finish()

wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


/opt/conda/envs/pt/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/pt/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
# let's free the GPU
model.to("cpu")
del model
torch.cuda.empty_cache()

## The Right way now!

We actually need to feed the model in a different way, with the right attention mask and penalising only the tokens on the answer:
- We will mask the question tokens
- We will put a large negative value on the labels we want to penalise
- Pre-tokenize the dataset for speed

In [ ]:
example = dataset[0]
example

Let's tokenize the prompt and the output separately:

In [ ]:
tokenized_prompt = tokenizer.encode(example["prompt"]) 
tokenized_output = tokenizer.encode(example["output"])

Now, let's set `input_ids` as the concatenation and `labels` as a copy of `input_ids` but masking the `prompt` 

In [ ]:
import copy
input_ids = tokenized_prompt + tokenized_output
labels = copy.deepcopy(input_ids)
labels[:len(tokenized_prompt)] = [-100] * len(tokenized_prompt)  # we mask the tokens from the prompt

So we will feed the model this `input_ids` and make it predict the `labels`, we also have to pass the corresponding attention mask so we don't attend to this tokens

Let's process the dataset now:

In [ ]:
def process_dataset(dataset):
    tokenized_dataset = []
    for example in progress_bar(dataset):
        tokenized_prompt = tokenizer.encode(example["prompt"]) 
        tokenized_output = tokenizer.encode(example["output"])
    
        input_ids = tokenized_prompt + tokenized_output
        labels = copy.deepcopy(input_ids)
        labels[:len(tokenized_prompt)] = [-100] * len(tokenized_prompt)
        tokenized_dataset.append({"input_ids": torch.tensor(input_ids), "labels": torch.tensor(labels)})
    return tokenized_dataset

In [ ]:
tokenized_dataset = process_dataset(dataset)

> You should save this tokenized dataset to W&B and reload from the artifact to save some time!

We now need to form batches, but the tokenized inputs have different sizes, so we will pad them

In [ ]:
tokenized_dataset[0]

In [ ]:
def collate_fn_tokenized(examples, pad_token=tokenizer.pad_token_id):
    input_ids, labels = tuple([example[key] for example in examples] for key in ("input_ids", "labels"))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=pad_token
    )
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    return dict(
        input_ids=input_ids,
        labels=labels,
        attention_mask=input_ids.ne(pad_token),
    )

So we pad labels with `-100` and the input_ids with the EOS token

In [ ]:
dummy_sample1 = {"input_ids":torch.tensor([1,2,3]), "labels":torch.tensor([-100,2,3])}
dummy_sample2 = {"input_ids":torch.tensor([5,6,7,8]), "labels":torch.tensor([-100,-100,7,8])}

collate_fn_tokenized([dummy_sample1, dummy_sample2], pad_token=-1)

In [ ]:
config.batch_size = 16

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=config.batch_size,
    collate_fn=collate_fn_tokenized,
    shuffle=True,
)

Let's create a fresh model =)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    # use_flash_attention_2=True,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16 if config.precision == "bf16" else torch.float32,
    use_cache=False
)


In [ ]:
n_freeze = 24

# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[n_freeze:].parameters(): param.requires_grad = True

In [ ]:
params, trainable_params = param_count(model)

In [ ]:
# Just freeze embeddings for small memory decrease
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False);

In [ ]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

In [ ]:
optim = torch.optim.Adam(model.parameters(), betas=(0.9,0.99), eps=1e-5)
scaler = torch.cuda.amp.GradScaler(enabled=(config.precision == "fp16")) # no-op if enabled=False
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.epoch_sz,
    num_warmup_steps=100,
)

In [ ]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["instruct"],
           config=config) # the Hyperparameters I want to keep track of

# Training
acc = TokenAccuracy()

model.train()
for step, batch in enumerate(progress_bar(train_dataloader)):
    optim.zero_grad(set_to_none=True)
    for micro_step in range(config.gradient_accumulation_steps):
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**to_gpu(batch))
            loss = out.loss / config.gradient_accumulation_steps
    scaler.scale(loss).backward()
    scaler.step(optim)
    scaler.update()
    scheduler.step()

    # we can log the metrics to W&B
    wandb.log({"loss": loss.item() * config.gradient_accumulation_steps,
               "accuracy": acc.update(out.logits, batch["labels"])})

    if step%config.log_every==0 or step%config.epoch_sz==0:
        prompt_table(eval_prompts, log=True)
    
# we save the model checkpoint at the end
if config.save_model:
    save_model(model, model_name=config.model_id.replace("/", "_"), models_folder="models/")
    
wandb.finish()

In [ ]:
!sudo poweroff